In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from hyperopt import tpe, anneal
import numpy as np

In [3]:
df = pd.read_csv('clean_imdb.csv')

In [4]:
pdf = df.reindex(np.random.permutation(df.index))

In [5]:
main_df = pd.read_csv('clean_data.csv')
X_test, y_test = main_df['clean_text'], main_df['class']

In [6]:
X, y = pdf['clean_text'], pdf['class']

In [7]:
vec = CountVectorizer(ngram_range=(1, 3))
X_enc = vec.fit_transform(X)
X_test_enc = vec.transform(X_test)

In [8]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
from sklearn.metrics import f1_score

def f1_loss(y_target, y_pred):
    f1 = f1_score(y_target, y_pred)
    print(f"this loss called with f1={f1}")
    return 1.0 - f1


from hpsklearn import HyperoptEstimator, any_classifier, any_preprocessing, any_sparse_classifier

np.random.seed(13)

estim = HyperoptEstimator(classifier=any_sparse_classifier('name'),
                          preprocessing=[],
                          algo=tpe.suggest,
                          max_evals=4,
                          trial_timeout=120,
                         loss_fn=f1_loss, verbose=False)

# Search the hyperparameter space based on the data

estim.fit(X_enc, y, random_state=42, cv_shuffle=False, n_folds=5, )

# Show the results

print(estim.score(X_enc, y))
# 1.0

print( estim.best_model() )

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely
100%|██████████| 1/1 [02:00<00:00, 120.30s/it, best loss: ?]


ValueError: attempt to get argmin of an empty sequence

In [ ]:
model = estim.best_model()['learner']
cross_val_score(model, X_test_enc, y_test, cv=5).mean()